# Amazon Reviews Sentiment Anslysis w/ Bert
By: Edward Zhang

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dalidevchallenge/amazon-neg.txt
/kaggle/input/dalidevchallenge/Sample - Superstore.csv
/kaggle/input/dalidevchallenge/amazon-pos.txt


In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2024-02-19 04:41:35.098290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 04:41:35.098410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 04:41:35.234013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
file_name_pos = "/kaggle/input/dalidevchallenge/amazon-pos.txt"
my_file = open(file_name_pos, "r") 
data = my_file.read() 
data_into_list = data.replace('\n', ' ').split(".")
my_file.close() 
pos_data = data_into_list
#pos_data = [i.strip() for i in data_into_list]
file_name_neg = "/kaggle/input/dalidevchallenge/amazon-neg.txt"
my_file = open(file_name_neg, "r") 
data = my_file.read() 
data_into_list = data.replace('\n', ' ').split(".")
my_file.close() 
neg_data = data_into_list
#neg_data = [i.strip() for i in data_into_list]

In [4]:
pos_data[4]

' But i guess can be long for corporate finance Good money'

In [5]:
pos_data[4]

' But i guess can be long for corporate finance Good money'

In [6]:
neg_data[7]

"  didn't last quite long enough  Opportunity takes time up to two plus years!  Long hours with only two 15 minute breaks and half hour lunch, so to have to stand on concrete without being able to sit is hard"

In [7]:
pos_df = pd.DataFrame(pos_data, columns=['review'])
pos_df = pos_df.assign(sentiment=[1]*len(pos_data))
neg_df = pd.DataFrame(neg_data, columns=['review'])
neg_df = neg_df.assign(sentiment=[0]*len(neg_data))

In [8]:
pos_df.head()

,review,sentiment
0,You're surrounded by smart people and the proj...,1
1,Brand name is great,1
2,Have yet to meet somebody who is unfamiliar w...,1
3,Hours weren't as bad as I had previously heard,1
4,But i guess can be long for corporate finance...,1


In [9]:
neg_df.head()

,review,sentiment
0,Internal tools proliferation has created a mes...,0
1,Most people are required to learn/understand ...,0
2,not the most stimulating work,0
3,Good brand name to work for but the work itse...,0
4,As a financial analyst you do very little fin...,0


In [10]:
combined_data = pd.concat([pos_df,neg_df])

In [11]:
combined_data = combined_data.sample(frac=1)


In [12]:
train=combined_data.sample(frac=0.8,random_state=200)
test=combined_data.drop(train.index)

In [13]:
train_y = train["sentiment"]
train_x = train.drop("sentiment", axis = 1)

test_y = test["sentiment"]
test_x = test.drop("sentiment", axis = 1)

In [14]:
x_test, x_val, y_test, y_val = train_test_split(test_x,
                                                    test_y,
                                                    test_size=0.2
                                                    )

# Bert

In [15]:
from transformers import BertTokenizer
model_id = "bert-base-uncased"
token = BertTokenizer.from_pretrained(model_id)
token

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [17]:
X_train_encoded = tokenizer.batch_encode_plus(train_x["review"].tolist(),
                                              padding=True, 
                                              truncation=True,
                                              return_tensors='tf')
 
X_val_encoded = tokenizer.batch_encode_plus(x_val["review"].tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              return_tensors='tf')
 
X_test_encoded = tokenizer.batch_encode_plus(x_test["review"].tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              return_tensors='tf')

In [18]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [20]:
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    train_y,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
    batch_size=32,
    epochs=10
)

Epoch 1/10


I0000 00:00:1708317756.101005      70 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/34 [==============================] - 76s 803ms/step - loss: 0.6224 - accuracy: 0.6516 - val_loss: 0.4639 - val_accuracy: 0.8333
Epoch 2/10
34/34 [==============================] - 25s 719ms/step - loss: 0.4004 - accuracy: 0.8512 - val_loss: 0.5862 - val_accuracy: 0.5833
Epoch 3/10
34/34 [==============================] - 25s 733ms/step - loss: 0.3062 - accuracy: 0.8785 - val_loss: 0.3649 - val_accuracy: 0.7500
Epoch 4/10
34/34 [==============================] - 25s 741ms/step - loss: 0.2130 - accuracy: 0.9303 - val_loss: 0.4297 - val_accuracy: 0.7500
Epoch 5/10
34/34 [==============================] - 26s 752ms/step - loss: 0.1340 - accuracy: 0.9557 - val_loss: 0.3317 - val_accuracy: 0.7500
Epoch 6/10
34/34 [==============================] - 26s 759ms/step - loss: 0.0923 - accuracy: 0.9708 - val_loss: 0.2268 - val_accuracy: 0.8333
Epoch 7/10
34/34 [==============================] - 26s 773ms/step - loss: 0.0644 - accuracy: 0.9802 - val_loss: 0.5799 - val_accuracy: 0.7500
Epoch 8/10

# Inference

In [21]:
pred = model.predict(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
 

2/2 [==============================] - 3s 89ms/step


In [22]:
logits = pred.logits
pred_labels = tf.argmax(logits, axis=1)
pred_labels = pred_labels.numpy()
label = {
    1: 'positive',
    0: 'Negative'
}
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]
 
print('Predicted Label :', pred_labels[:10])
print('Actual Label    :', Actual[:10])

Predicted Label : ['positive', 'positive', 'Negative', 'Negative', 'positive', 'Negative', 'positive', 'Negative', 'positive', 'Negative']
Actual Label    : ['positive', 'Negative', 'Negative', 'Negative', 'positive', 'Negative', 'positive', 'Negative', 'positive', 'positive']


In [23]:
x_test["review"].tolist()[3]

' Not good for people with brains'

In [24]:
pred_labels[3], y_test.tolist()[3]

('Negative', 0)